In [ ]:
import pandas as pd

In [ ]:
#demo = pd.read_csv("../Data/raw/df_final_demo.txt", delimiter=",") 

In [ ]:
demo = pd.read_csv("../Data/raw/df_final_demo.txt", delimiter=",") 

In [ ]:
demo.head()

In [ ]:
print(demo.shape)

demo.columns = (
    demo.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)

In [ ]:
demo.isnull().sum()

In [ ]:
demo = demo.dropna(subset=['bal'])
demo.isnull().sum()

In [ ]:
print(demo.shape)

In [ ]:
#Check duplicates client_id   --------------------------   CLIENT_ID -------------------------
duplicated = demo['client_id'].duplicated().sum()
print(f"Duplicates clients: {duplicated}")

In [ ]:
#Cleaning the clnt_tenure_yr column ----------------------- clnt_tenure_yr--------------------

# Convert to integers and correct out-of-range values (0–50)
demo['clnt_tenure_yr'] = demo['clnt_tenure_yr'].round().astype('Int64')
demo.loc[~demo['clnt_tenure_yr'].between(0, 50), 'clnt_tenure_yr'] = pd.NA

In [ ]:
demo['clnt_tenure_yr'].max()

In [ ]:
#Cleaing the clnt_tenure_mnth column ----------------------------clnt_tenure_mnth-----------------------------------------

# Round and convert to integers
demo['clnt_tenure_mnth'] = demo['clnt_tenure_mnth'].round().astype('Int64')

# If there are months >= 12, convert the excess to years
overflow = demo['clnt_tenure_mnth'] // 12
demo['clnt_tenure_yr'] += overflow
demo['clnt_tenure_mnth'] = demo['clnt_tenure_mnth'] % 12

In [ ]:
#Cleaning the clnt_age column -------------------------------clnt_age------------------------------------

# Fill the single null with the median
median_age = demo['clnt_age'].median()
demo['clnt_age'] = demo['clnt_age'].fillna(median_age).round().astype('Int64')

# Nullify ages outside of 18–100
demo.loc[~demo['clnt_age'].between(18, 100), 'clnt_age'] = pd.NA

In [ ]:
#Cleaning the gender column --------------------------------------gendr-----------------------------------------
#use dictionary for map
map_gender = {
    'm': 'male',
    'male': 'male',
    'f': 'female',
    'female': 'female'
}
#convert to object and nulls are unknow:
demo['gendr'] = (
    demo['gendr']
        .astype(str)
        .str.strip()
        .str.lower()
        .map(map_gender)
        .fillna('unknown')
)

In [ ]:
print(demo)

In [ ]:
## The data look very clean. We need only to change the datatype of columns: num_accts, calls_6_mnth, logons_6_mnth
import numpy as np
demo["num_accts"] = demo["num_accts"].fillna(0).astype(int)
demo['num_accts'] = demo['num_accts'].apply(np.int64)
demo["calls_6_mnth"] = demo["calls_6_mnth"].fillna(0).astype(int)
demo['calls_6_mnth'] = demo['calls_6_mnth'].apply(np.int64)
demo["logons_6_mnth"] = demo["logons_6_mnth"].fillna(0).astype(int)
demo['logons_6_mnth'] = demo['logons_6_mnth'].apply(np.int64)

In [ ]:
print(demo)

In [ ]:
from pathlib import Path


Path("data/interim").mkdir(parents=True, exist_ok=True)

demo.to_csv("data/interim/demo_clean.csv", index=False)

print("✔️ Archivo guardado en data/interim/demo_clean.csv")

In [ ]:
# unique values logons
demo.logons_6_mnth.unique()

In [ ]:
# median and mean logons
display(demo.logons_6_mnth.median())
display(demo.logons_6_mnth.mean())

In [ ]:
# new df with users over median
demo_primary = demo[demo["logons_6_mnth"] > 6]
demo_primary.logons_6_mnth.unique()

In [ ]:
# How many percent of users are top users?
# What is the average of usage overall? How many users do 0, 1-3, 4-6, 7-9

In [ ]:
# tenure of top users
demo_primary.tenure = (((demo_primary.clnt_tenure_yr * 12) + demo_primary.clnt_tenure_mnth)/12)
demo.tenure = (((demo.clnt_tenure_yr * 12) + demo.clnt_tenure_mnth)/12)

In [ ]:
# tenure of top users
display(demo_primary.tenure.median())
display(demo_primary.tenure.mean())

display(demo.tenure.median())
display(demo.tenure.mean())

In [ ]:
# Client Age of top users
display(demo_primary.clnt_age.describe())
display(demo.clnt_age.describe())

In [ ]:
# Gender of top users
grouped = demo_primary.groupby('gendr').size()
grouped_percentage = (grouped / grouped.sum()) * 100
grouped_percentage_df = grouped_percentage.reset_index(name='percentage')
print(grouped_percentage_df)

grouped = demo.groupby('gendr').size()
grouped_percentage = (grouped / grouped.sum()) * 100
grouped_percentage_df = grouped_percentage.reset_index(name='percentage')
print(grouped_percentage_df)

In [ ]:
# number of accounts
display(demo_primary.num_accts.describe())
display(demo.num_accts.describe())

In [ ]:
# balance
display(demo_primary.bal.mean())
display(demo_primary.bal.median())
display(demo.bal.mean())
display(demo.bal.median())